In [1]:
!pip install shap
import shap

In [2]:
data,labels =  shap.datasets.adult()

In [3]:
data

,Age,Workclass,Education-Num,Marital Status,Occupation,Relationship,Race,Sex,Capital Gain,Capital Loss,Hours per week,Country
0,39.0,7,13.0,4,1,0,4,1,2174.0,0.0,40.0,39
1,50.0,6,13.0,2,4,4,4,1,0.0,0.0,13.0,39
2,38.0,4,9.0,0,6,0,4,1,0.0,0.0,40.0,39
3,53.0,4,7.0,2,6,4,2,1,0.0,0.0,40.0,39
4,28.0,4,13.0,2,10,5,2,0,0.0,0.0,40.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27.0,4,12.0,2,13,5,4,0,0.0,0.0,38.0,39
32557,40.0,4,9.0,2,7,4,4,1,0.0,0.0,40.0,39
32558,58.0,4,9.0,6,1,1,4,0,0.0,0.0,40.0,39
32559,22.0,4,9.0,4,1,3,4,1,0.0,0.0,20.0,39


In [4]:
data.shape

(32561, 12)

In [5]:
labels

array([False, False, False, ..., False, False,  True])

In [6]:
labels.shape

(32561,)

In [8]:
import lightgbm  as lgbm

In [9]:
params = {'objective':'binary',
          'booster_type':'gbdt',
          'max_depth':6,
          'learning_rate':0.05,
          'metric':'auc'
}

In [10]:
dtrain = lgbm.Dataset(data,labels)
model = lgbm.train(params,dtrain,num_boost_round=100,valid_sets=[dtrain])


[LightGBM] [Warning] Unknown parameter: booster_type
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Find whitespaces in feature_names, replace with underlines
[LightGBM] [Warning] Unknown parameter: booster_type
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 7841, number of negative: 24720
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000803 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 431
[LightGBM] [Info] Number of data points in the train set: 32561, number of used features: 12
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.240810 -> initscore=-1.148246
[LightGBM] [Info] Start training from score -1.14

In [11]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

predicted = model.predict(data)
bool_predicted = predicted>0.5

print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy_score(labels, bool_predicted)))

cm = confusion_matrix(labels, bool_predicted)
print(cm)

print('\nTrue Positives(TP) = ', cm[0,0])
print('\nTrue Negatives(TN) = ', cm[1,1])
print('\nFalse Positives(FP) = ', cm[0,1])
print('\nFalse Negatives(FN) = ', cm[1,0])


LightGBM Model accuracy score: 0.8752
[[23521  1199]
 [ 2864  4977]]

True Positives(TP) =  23521

True Negatives(TN) =  4977

False Positives(FP) =  1199

False Negatives(FN) =  2864


In [12]:
model.save_model('saved_adult_model.txt')